In [1]:
pip install transformers datasets evaluate accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1

In [2]:
from google.colab import drive
import datetime
from datasets import (load_dataset,
                      Dataset,
                      DatasetDict)
import pandas as pd
import torch
from transformers import (AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          AutoTokenizer,
                          set_seed)
from tqdm.auto import tqdm
from peft import PeftModel

import accelerate


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model = "microsoft/phi-2"
model_name = model.replace('.','').replace('-','_').replace('/','_')
filestamp = "1711835043" # Select the timestamp of the latest version
timestamp = "1715888640" # Select the timestamp of the latest model
checkpoint = "checkpoint-18000"
source = "mix"
test_path = f'/content/drive/MyDrive/Thesis data/relevant_selection/{filestamp}_{source}_test.json'
base_result_path = f'/content/drive/MyDrive/Thesis data/PHI_results/{filestamp}_{source}_q{model_name}.csv'
set_seed(42)
adapter_name = f"{timestamp}_{filestamp}_{source}_trained"
input_dir = f'/content/drive/MyDrive/Thesis_models/{adapter_name}/{checkpoint}'
adapter_result_path = f'/content/drive/MyDrive/Thesis data/PHI_results/{filestamp}_{source}_q{model_name}_adapter.csv'

In [5]:
# data loading
test = pd.read_json(test_path)

test_ds = Dataset.from_pandas(test)

data = DatasetDict()
data['test'] = test_ds

del test, test_ds

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model,trust_remote_code=True,)
# set pad token - to avoid error while training
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [8]:
# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map='auto',
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
def response_generator(model, tokenizer, data):
  responses = []
  for row in tqdm(data['test'], total=data['test'].num_rows):
    input_ids = tokenizer(row['prompt'], truncation=True, return_tensors='pt').input_ids.to("cuda:0")
    output = model.generate(input_ids, max_new_tokens=1, pad_token_id = 50256)
    response = tokenizer.batch_decode(output)
    result = response[0].split('\n\nOutput:')[1]
    # result = result.replace(' ','')
    result = result.replace('\n','')
    responses.append(result)
  return responses

In [10]:
# Get the results for the quantized base model
results = pd.DataFrame(response_generator(model, tokenizer, data), columns=['base'])
results

  0%|          | 0/200 [00:00<?, ?it/s]

,base
0,UP
1,Up
2,Down
3,
4,The
...,...
195,
196,Up
197,Down
198,Up


In [11]:
results.to_csv(base_result_path)

In [12]:
# Load the trained model
peft_model = PeftModel.from_pretrained(model,input_dir,is_trainable=False)

In [13]:
# Get the results for the quantized trained model
results = pd.DataFrame(response_generator(peft_model, tokenizer, data), columns=['trained'])
results

  0%|          | 0/200 [00:00<?, ?it/s]

,trained
0,UP
1,DOWN
2,UP
3,DOWN
4,DOWN
...,...
195,UP
196,DOWN
197,UP
198,DOWN


In [14]:
results.to_csv(adapter_result_path)